A model for generating SMILES strings from images of molecules.

In [1]:
## setup cell
# import required libraries
import numpy as np
import pandas as pd
import csv
import math
import PIL
from PIL import ImageOps
from sklearn import model_selection
import keras as keras
from keras import layers
from keras import regularizers
from keras import optimizers

# load the list of compounds and image conversion errors
compounds = pd.read_csv("compounds.csv")
with open("errors.csv", "r") as file:
    errors = list(csv.reader(file))
errors = list(map(int, errors[0]))

# filter out the compounds that threw errors
compounds = compounds.query("SubstanceID not in @ errors")

# generate a dictionary of unique characters in SMILES strings and its inverse
characters = sorted(list(set(compounds["SMILES"].sum())))
character_dictionary = dict((j, i) for i, j in enumerate(characters))
character_dictionary_inverse = dict((i, j) for i, j in enumerate(characters))

# augment the dictionary with start, stop, and "blank" codons
character_dictionary.update({"start": 68, "stop": 69, "blank": 70})
character_dictionary_inverse.update({68: "start", 69: "stop", 70: ""})

Using TensorFlow backend.
/home/ubuntu/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ubuntu/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ubuntu/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/ubuntu/.local/lib/python3.6/site-packages/tensorflow/python

In [2]:
## to keep training time reasonable, we will truncate the dataset
# subset dataset to ~50,000 shortest SMILES strings
compounds["SMILES_length"] = compounds.SMILES.map(lambda x: len(x))
cutoff_length = compounds["SMILES_length"].quantile(50000 / len(compounds))
compounds_subset = compounds.query("SMILES_length <= @cutoff_length")

# randomly sample 15,000 compounds
compounds_subset = compounds_subset.sample(15000, random_state = 738523)

# find the longest SMILES string in the dataset; add space for start and end codons
max_length = compounds_subset["SMILES"].map(lambda x: len(x)).max() + 2

In [3]:
## test, train, validation, split
# split 80/20 into train and test sets
train, test = model_selection.train_test_split(compounds_subset, test_size = 0.2, random_state = 1935235)

# further sudivide training set into train and validation sets to give an 80/20/20 overall split
train, validation = model_selection.train_test_split(train, test_size = 0.25, random_state = 358235)

In [4]:
## a simple function to pad encoded strings with blanks
# declare a function pad
# string: a list generated by encoding a string with character_dictionary
# max_length: the number of items to pad the string out to
# returns: a list of integers padded with 70 to length max_length

def pad(string, max_length):
    while len(string) < max_length:
        string.append(70)
    return(string)

In [5]:
## data generator to supply images and targets to the neural network
# declare a function molecule_image_generator
# SubstanceIDs: a list of SubstanceIDs from which to draw batches
# image_batch_size: the number of images included in each "batch"
# batch_size: the number of rows to return in each batch
# returns: a data generator

def molecule_image_generator(SubstanceIDs, image_batch_size, batches_per_image):
    # calculate the maximum number of batches from one iteration of SubstanceIDs
    max_batches = math.floor(len(SubstanceIDs) / image_batch_size)

    # declare a counter to track current batch
    batch = 0
    
    # loop through batches indefinitely
    while True:                
        # declare an empty array to hold the image data
        # shape == (image_batch_size * max_length), xpixels, ypixels, channels
        X1 = np.empty([image_batch_size * (max_length), 300, 300, 1], dtype = "int8")
        
        # declare an empty array to hold the SMILES string input fragments 
        # shape == (image_batch_size * (max_length), max_length
        X2 = np.empty([image_batch_size * (max_length), max_length], dtype = "int8")
        
        # declare an empty array (vector) to hold the SMILES string target fragment
        # shape == image_batch_size * max_length
        Y = np.empty([image_batch_size * max_length], dtype = "int8")
        
        
        # determine which images to place in this batch
        # take image_batch_size observations if possible
        if batch < max_batches:
            batch_start = batch * image_batch_size
            batch_end = batch * image_batch_size + image_batch_size
            batch += 1
        # otherwise take as many observations as possible 
        else:
            batch_start = batch * image_batch_size
            batch_end = batch * image_batch_size + (len(SubstanceIDs) % image_batch_size)
            batch = 0
        
        
        # generate X1, X2, Y for a batch of images
        # loop through images by SubstanceID index
        for i in range(batch_start, batch_end):
            # encode the SMILES string for the current SubstanceID into a list of integers
            SMILES = compounds_subset.query("SubstanceID == @SubstanceIDs[@i]")["SMILES"].to_string(header = False, index = False).strip()
            SMILES = [character_dictionary[character] for character in SMILES]
            
            # add start and stop codons, pad with blanks to max_length
            SMILES.insert(0, 68)
            SMILES.append(69)
            SMILES = pad(SMILES, max_length)
            
            # loop through each character in the SMILES string
            for j in range(0, len(SMILES)):
                # calcuate the current observation number
                obs = (i - batch_start) * len(SMILES) + j
                
                # import image, invert to a black background, white structure
                image = PIL.Image.open("./molecule_drawings/" + str(SubstanceIDs[i]) + ".png")
                image = image.convert("L")
                image = ImageOps.invert(image)
                image = image.convert("1")
                image = np.array(image, dtype = "int8")
                
                # add image to X1; augment with a single channel dimension
                X1[obs, :, :, 0] = image
                
                # split SMILES at each character into an initiating string and a target character
                X2[obs, :] = pad(SMILES[:j], max_length)
                Y[obs] = SMILES[j]
                
        
        # one-hot encode Y
        Y = keras.utils.to_categorical(Y)

        # subset X1, X2, Y into batches_per_image arrays
        X1 = np.array_split(X1, batches_per_image)
        X2 = np.array_split(X2, batches_per_image)
        Y = np.array_split(Y, batches_per_image)
        
        # yield the generated data to the training function in batches of batch_size observations
        for i in range(batches_per_image):
            yield([[X1[i], X2[i]], Y[i]])

Our network will take two inputs:  
1) The images, which are run through a convolutional branch.  
2) The growing SMILES strings, which are run through an LSTM branch.

The two inputs are merged into an FFNN trunk with softmax output.

In [6]:
## define the neural network architecture
# regularization parameters
l2_lambda = 0.01
lambda_RNN = 0.02
dropout_rate = 0

# CNN layers for images
input_image = layers.Input(shape = (300, 300, 1))
conv_1 = layers.Conv2D(
    filters = 64, 
    kernel_size = (3, 3), 
    strides = (1, 1), 
    activation = "relu",
    kernel_regularizer = regularizers.l2(l2_lambda)
)(input_image)
max_pool_1 = layers.MaxPool2D(pool_size = (3, 3))(conv_1)
conv_2 = layers.Conv2D(
    filters = 64, 
    kernel_size = (5, 5), 
    strides = (1, 1), 
    activation = "relu",
    kernel_regularizer = regularizers.l2(l2_lambda)
)(max_pool_1)
max_pool_2 = layers.MaxPool2D(pool_size = (5, 5))(conv_2)
flatten = layers.Flatten()(max_pool_2)
dense_CNN = layers.Dense(
    units = 32,
    activation = "relu",
    kernel_regularizer = regularizers.l2(l2_lambda)
)(flatten)
dropout_CNN = layers.Dropout(rate = dropout_rate)(flatten)

# RNN layers for SMILES strings
input_SMILES = layers.Input(shape = (max_length, ))
embedding = layers.Embedding(input_dim = max_length, output_dim = 200)(input_SMILES)
lstm = layers.LSTM(
    units = 32,
    kernel_regularizer = regularizers.l1(lambda_RNN),
    #recurrent_regularizer = regularizers.l2(lambda_RNN),
    activity_regularizer = regularizers.l1(lambda_RNN)
)(embedding)

# merge into FFNN layers
sum_inputs = layers.add([dense_CNN, lstm])
dense_output = layers.Dense(
    units = 256, 
    activation = "relu",
    kernel_regularizer = regularizers.l2(l2_lambda)
)(sum_inputs)
dropout_output = layers.Dropout(rate = dropout_rate)(dense_output)
output = layers.Dense(units = len(character_dictionary), activation = "softmax")(dropout_output)

# combine inputs and outputs into model
model = keras.Model(inputs = [input_image, input_SMILES], outputs = output)

# compile the model
model.compile(
    loss = "categorical_crossentropy", 
    optimizer = optimizers.adam(lr = 1e-4, decay = 1e-8), 
    metrics = ["accuracy"]
)

Instructions for updating:
Colocations handled automatically by placer.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 300, 300, 1)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 298, 298, 64) 640         input_1[0][0]                    
__________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)  (None, 99, 99, 64)   0           conv2d_1[0][0]                   
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 95, 95, 64)   102464      max_pooling2d_1[0][0]            
_____________________________________

In [7]:
## train the model
# set the number of epochs
epochs = 5
starting_epoch = 1
# set the number of images to process at once
image_batch_size = 3
# set the number of batches per image
batches_per_image = 1

# train the model
model.fit_generator(
    generator = molecule_image_generator(list(train.SubstanceID), image_batch_size, batches_per_image), 
    steps_per_epoch = (len(train) / image_batch_size) * batches_per_image, 
    validation_data = molecule_image_generator(list(validation.SubstanceID), image_batch_size, batches_per_image), 
    validation_steps = (len(validation) / image_batch_size) * batches_per_image, 
    nb_epoch = epochs,
    initial_epoch = starting_epoch
)

Instructions for updating:
Use tf.cast instead.


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(generator=<generator..., steps_per_epoch=3000.0, validation_data=<generator..., validation_steps=1000.0, epochs=1)`


Epoch 1/1
3000/3000 [==============================] - 2956s 985ms/step - loss: 28.7524 - acc: 0.3672 - val_loss: 23.5767 - val_acc: 0.3772
